In [1]:
#Imports

import pandas as pd
import numpy as np
import plotly.offline as plt
import plotly.graph_objs as go
import os
import random
import math
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr
import theano
from multiprocessing import Pool, cpu_count

ModuleNotFoundError: No module named 'theano'

In [2]:
#Data Location

DATA_DIR = "./data/"
SONGS_FILE = "songs.csv"

In [3]:
#Data Features

NFEATURE = 21 #Number of Features
S = 50 #Hyper Parameter
totReco = 0 #Number of total recommendation till now
startConstant = 5 #for low penalty in starting phase

In [5]:
#Read Data

Songs = pd.read_csv(DATA_DIR + SONGS_FILE, index_col=0)

In [7]:
def compute_utility(user_features, song_features, epoch, s=S):
    
    """ Compute utility U based on user preferences and song preferences """
    
    user_features = user_features.copy()
    song_features = song_features.copy()
    
    dot = user_features.dot(song_features)
    ee = (1.0 - 1.0*math.exp(-1.0*epoch/s))
    
    res = dot * ee
    
    return res

In [8]:
def get_song_features(song):
    
    """ Feature of particular song """
    
    if isinstance(song, pd.Series):
        return song[-NFEATURE:]
    
    elif isinstance(song, pd.DataFrame):
        return get_song_features(pd.Series(song.loc[song.index[0]]))
    
    else:
        raise TypeError("{} should be a Series or DataFrame".format(song))

In [9]:
def best_recommendation(user_features, epoch, s):
    
    global Songs
    Songs = Songs.copy()
    
    """ Song with highest utility """
    
    utilities = np.zeros(Songs.shape[0])
    
    for i, (Title, song) in enumerate(Songs.iterrows()):
        song_features = get_song_features(song)
        utilities[i] = compute_utility(user_features, song_features, epoch - song.last_t, s)
        
    return Songs[Songs.index == Songs.index[utilities.argmax()]]

In [10]:
def all_recommendation(user_features):
    
    """ Top 10 songs with using exploration and exploitation """
    
    global Songs
    Songs = Songs.copy()
    
    i=0
    
    recoSongs = []
    
    while i < 10:
        song = greedy_choice_no_t(user_features, totReco, S)
        recoSongs.append(song)
        Songs.loc[Songs.index.isin(song.index),'last_t'] = totReco
        i += 1
        
    return recoSongs